### Reinforcment Learning

El aprendizaje por refuerzo, es una rama del **aprendizaje automatica**. Estudia **agentes** que deben aprender a tomar **decisiones secuenciales** en un **entorno estocastico**, con el objetivo de maximizar una **recompensa acumulada** a largo plazo.

Finalmente RL se modelo de forma formal como un MDP (Markov Decision Process).

#### Secuencia de eventos en RL

1. El agente observa el estado $s_0$ del entorno.
2. Elige una accion $a_0$ basada en su politica actual $\pi$ (por ejemplo, una politica epsilon-greedy).
3. La accion $a_0$ es ejecutada en el entorno.
4. El agente observa el nuevo estado $s_1$ segun la funcion $P_a(s,s') = s_1$.
5. El agente recibe una recompensa $r_1$ segun la funcion $R_a(s,s') = r_1$.
6. El agente actualiza su politica $\pi$ para maximizar la recompensa futura esperada.


### Q-Learning y MDP

"For any finite Markov decision process, Q-learning finds an optimal policy, maximizing the expected value of the total reward over any and all successive steps, starting from the current state".

Es una aplicacion basada en un MDP, donde el agente aprende una politica optima sin conocer las funciones de transicion $P_a(s,s')$ y recompensa $R_a(s,s')$ del MDP. El agente aprende a traves de la interaccion con el entorno. Es una aplicacion especifica de RL. Pertenece a la familia off-policy.

- El agente aprende una funcion de valor de accion $Q(s,a)$ (ecuacion de Bellman), que estima la recompensa futura esperada al tomar la accion $a$ en el estado $s$ y seguir la politica optima a partir de ahi.

$$
Q^*(s,a) = \mathbb{E}\big[ r_{t+1} + \gamma \max_{a'} Q^*(s_{t+1}, a') ,\big|, s_t = s, a_t = a \big]
$$

- $r_{t+1} :=$ recompensa inmediata

El algoritmo actualiza una tabla ($Q(s,a)$):

$$
\text{error temporal(TD err)} = r_{t+1} + \gamma \max_{a'} Q(s_{t+1}, a') - Q(s_t,a_t) = \text{(realidad observada)-(valor previo)} = \delta_t 
$$
$$
Q(s_t,a_t) \leftarrow Q(s_t,a_t) + \alpha \big[\delta_t  \big]
$$

- Pero esto solo funciona si los conjuntos (S) y (A) son **pequeños y discretos**, ya que guardar una tabla (Q) completa es inviable en la práctica.

###### intuicion: Q* es la funcion de politica optima que nos gustaria conocer, esta es el valor esperado mas la recompensa inmediata, mas las recompensas futuras esperadas con menos peso que el presente. El agente no conoce Q*, pero si conoce la recompensa inmediata y el valor de la siguiente accion, por lo que puede aproximar Q* con la ecuacion de TD error, que es basicamente el aposteriori y apriori restados. El agente actualiza su valor Q(s,a) con un paso de aprendizaje alpha, para acercarse a Q*. Con el tiempo y muchas iteraciones, Q(s,a) converge a Q*. 

#### Off-policy vs On-policy



| **Aspecto**                            | **On-policy**                                                                                                                                                    | **Off-policy**                                                                                                                                                          |
| -------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Definición general**                 | El agente **aprende sobre la política que realmente ejecuta** en el entorno.                                                                                     | El agente **aprende sobre una política diferente** de la que usa para generar experiencias.                                                                             |
| **Política de comportamiento** (($\mu$)) | Es la **misma** que la política objetivo (($\pi$)):  ($\mu = \pi$).                                                                                                  | Es **distinta** de la política objetivo:  ($\mu \neq \pi$).                                                                                                               |
| **Política objetivo (($\pi$))**          | Se mejora usando los datos generados por ella misma.                                                                                                             | Se aprende una política óptima ($\pi^*$) mientras se comporta de otra forma (por ejemplo, exploratoria).                                                                  |
| **Exploración vs explotación**         | Debe equilibrar ambos aspectos dentro de la misma política (por ejemplo, usando ($\epsilon$)-greedy).                                                              | Puede explorar libremente con una política exploratoria ($\mu$), mientras aprende una política óptima determinista ($\pi^*$).                                               |
| **Ejemplo clásico**                    | SARSA (State–Action–Reward–State–Action).                                                                                                                        | Q-Learning.                                                                                                                                                             |
| **Actualización típica**               | Usa la acción **realmente ejecutada** en el siguiente estado:  [ $Q(s_t,a_t) \leftarrow Q(s_t,a_t) + \alpha [r_{t+1} + \gamma Q(s_{t+1}, a_{t+1}) - Q(s_t,a_t)$] ] | Usa la **mejor acción posible** según el valor máximo estimado:  [ $Q(s_t,a_t) \leftarrow Q(s_t,a_t) + \alpha [r_{t+1} + \gamma \max_{a'} Q(s_{t+1}, a') - Q(s_t,a_t)$] ] |
| **Consecuencia**                       | Aprende la política **real que sigue** el agente, lo que puede hacer el aprendizaje más estable pero menos óptimo.                                               | Aprende la política **óptima** incluso si el comportamiento durante el entrenamiento no es óptimo.                                                                      |
| **Ventaja principal**                  | Mejor correlación entre datos y política aprendida (menor varianza).                                                                                             | Mayor eficiencia en el aprendizaje y mejor convergencia hacia la política óptima.                                                                                       |
| **Desventaja principal**               | Puede quedarse atrapado en políticas subóptimas si la exploración no es suficiente.                                                                              | Mayor varianza y posible inestabilidad si la política exploratoria ($\mu$) difiere mucho de ($\pi$).                                                                        |
| **Interpretación intuitiva**           | “Aprendo lo que hago.”                                                                                                                                           | “Aprendo lo que **debería** hacer, no necesariamente lo que **estoy** haciendo.”                                                                                        |


* En *on-policy*, el aprendizaje sigue la distribución real de los datos.

La expectativa de actualización se toma **bajo la misma distribución** de política:
$$
\mathbb{E}_{s,a \sim d^\pi} [\delta_t]
$$

* En *off-policy*, el aprendizaje requiere **corregir** o **reponderar** las muestras si se quiere una estimación no sesgada (a menudo mediante *importance sampling*).


La expectativa de actualización se toma **bajo otra distribución de datos**:
$$
\mathbb{E}_{s,a \sim d^\mu} [r + \gamma \mathbb{E}_{a'\sim \pi}[Q(s',a')] - Q(s,a)]
$$

donde ($d^\mu(s,a)$) es la distribución estacionaria inducida por la política de comportamiento.





